# Computer Vision 

# Exercise 7: Feature Matching

- TU Chemnitz
    - Fak. für Informatik
        - Professur Künstliche Intelligenz
            - Lehre
                - Bildverstehen
     
Contact:
* julien dot vitay at informatik dot tu-chemnitz dot de
* abbas dot al-ali at informatik dot tu-chemnitz dot de

Course web page:
[https://www.tu-chemnitz.de/informatik/KI/edu/biver/](https://www.tu-chemnitz.de/informatik/KI/edu/biver/)

### Harris Corner detection

<img src="img/castle-harris.png" alt="img/castle-harris.png" width="400"/>   

1. Compute the gradient at each location:

$$
    \mathbf{\nabla} I(x, y) = \begin{bmatrix} \frac{\partial I(x , y)}{\partial x} &\frac{\partial I(x , y)}{\partial y} \\ \end{bmatrix} = \begin{bmatrix} I_x & I_y \\ \end{bmatrix}
$$

2. Compute the auto-correlation matrix at each location by using a window function: 

$$
    \mathbf{A} = \sum_{x, y} w(x, y)  \begin{bmatrix} I_x^2 & I_x I_y \\ I_x I_y & I_y^2\\ \end{bmatrix} = w \ast \begin{bmatrix} I_x^2 & I_x I_y \\ I_x I_y & I_y^2\\ \end{bmatrix}
$$

3. Determine the Harris score at each location:

$$
    f(x,y) = \text{det}(\mathbf{A}) - \kappa \,  \text{tr}(\mathbf{A})^2
$$

4. Threshold the Harris score to keep only the most interesting points.

5. Suppress the local non-maxima to keep a single point in each blob.


* The three first steps of Harris corner detection are provided by the `cv2.cornerHarris()` method:

```python
score = cv2.cornerHarris(img, blockSize=2, ksize=3, k=0.04)
```

* `img` must be a grayscale 32-bits floating-point representation of the image (`gray.astype(np.float32)`).

* `blockSize` defines the size of the window function.

* `ksize` is the size of the Sobel filters used to compute the gradients.

* `k` is the $\kappa$ parameter of the Harris score function.

### **Task 1:** Harris corner detection

1. Visualize the Harris score of the images `star.jpg` and `castle.jpg`. Display the range of values by using `plt.colorbar()`.

2. Vary the `blockSize` and `ksize` parameters. Conclude on their effect on corner detection.

3. The Harris score has to be thresholded. A rule of thumb is to keep only scores higher than 1% of the maximum score (`score.max()`). Use `cv2.threshold` on the Harris score and visualize the result. 

4. Vary the threshold (0.1%, 10%...) and conclude on its importance for corner detection. 

- Several methods are available for non-maxima suppression. Here we will compute the center of each **blob** of points in the thresholded score image: 

```python
_, _, _, corners = cv2.connectedComponentsWithStats(thresh_score.astype(np.uint8))
```

5. Print the number of the corners (use the `shape` of `corners`). Visualize the corners. Add them the original image to see to what they correspond.

6. Now that you have the complete Harris detector, vary again the various parameters (`blockSize`, `ksize`, `k` and the threshold percentage) to see how they influence corner detection. Search for corners in other images.


**Note:** it would be a better practice to refine the position of the corners using `cv2.cornerSubPix`:

```python
_, _, _, centroids = cv2.connectedComponentsWithStats(thresh_score.astype(np.uint8))
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray, centroids.astype(np.float32), (5,5), (-1, -1), criteria)
```

In [ ]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import cv2

In [ ]:
# Harris

# Load the image 
img = cv2.imread('castle.jpg')
#img = cv2.imread('star.jpg')

# Convert to float32 gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)

# Compute the Harris score
score = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)

# Show the score
plt.figure()
plt.imshow(score, cmap=plt.cm.gray)
plt.xticks([]); plt.yticks([])
plt.colorbar()
plt.title('Harris Score')
plt.show()

# Threshold the score
ret, thresh_score = cv2.threshold(score, 0.01*score.max(), 255, cv2.THRESH_BINARY)

plt.figure()
plt.imshow(thresh_score, cmap=plt.cm.gray)
plt.xticks([]); plt.yticks([])
plt.title('Thresholded Harris Score')
plt.show()

# Non-maxima suppression using centroids
_, _, _, corners = cv2.connectedComponentsWithStats(thresh_score.astype(np.uint8))
print(corners.shape[0], 'corners detected.')

"""
_, _, _, centroids = cv2.connectedComponentsWithStats(thresh_score.astype(np.uint8))
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray, centroids.astype(np.float32), (5,5), (-1, -1), criteria)
print(corners.shape[0], 'corners detected.')
"""

plt.figure()
for c in range(corners.shape[0]):
    plt.plot(corners[c, 0], corners[c, 1], '.b')
plt.xticks([]); plt.yticks([])
plt.xlim((0, img.shape[1]-1))
plt.ylim((img.shape[0]-1, 0))
plt.title('Non-maxima suppression')
plt.show()

# Plot the result
plt.figure()
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
for c in range(corners.shape[0]):
    plt.plot(corners[c, 0], corners[c, 1], '.b')
plt.xticks([]); plt.yticks([])
plt.title('Harris detection')
plt.show()

### Template matching

<img src="img/matching.png" alt="img/matching.png" width="600"/> 

### Overview of feature matching

 <figure>
  <img src="img/featurematching.png" alt="img/featurematching.png" width="400"/> 
  <figcaption>Source: J. Hays</figcaption>
</figure> 

**1 - Feature detection**

1. Find a set of interest points or **features**.

2. Define a region around each point.

$\rightarrow$ *Harris Corner detection, LoG detection, Suboctave DoG pyramid...*

**2 - Feature description**

3. Extract and normalize the region content.

4. Compute a local descriptor.

$\rightarrow$ *SIFT, SURF, GLOH, ORB, HOG, PCA...*

**3 - Feature matching**

5. Match local descriptors between images. 

$\rightarrow$ *Nearest neighbors, K-d trees...*

**4 - Feature alignment**

6. Estimate the transformation between the features. 

$\rightarrow$ *Least squares, RANSAC...*

### Steps of template matching

**1 - Feature detection**

* Detect interest points in the *template* and *scene* images using a **Suboctave DoG pyramid**.

**2 - Feature description**

* Compute the **ORB** representation of the features.

**3 - Feature matching**

* Match the features of the template and scene using **Nearest neighbor distance ratio matching**.

**4 - Feature alignment**

* Estimate the transformation and reject outliers with **RANSAC**.

### ORB feature detection and description

* **ORB** (Oriented FAST and Rotated BRIEF) is a fast and free alternative to SIFT and SURF feature descriptors.

<http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html>

* Interest points are detected using the FAST algorithm:

<http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_fast/py_fast.html>

* Feature descriptions are computed using the BRIEF (Binary Robust Independent Elementary Features) method:

<http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_brief/py_brief.html>

* In the end, the only differences with SIFT are:

    1. A much faster algorithm.

    2. 32-bits descriptors instead of 128-bits.

    3. A patent-free method that works almost as well as SIFT.

### **Task 2:**  Feature detection and ORB description

```python
template = cv2.imread('beer.png',0)
scene = cv2.imread('scene.jpg',0)
```

* The OpenCV ``cv2.ORB_create()`` method creates a ORB object that both **detects** interest points using Suboctave DoG pyramids and **computes** a description vector.


```python
orb = cv2.ORB_create(nfeatures=1500, scaleFactor=1.3, WTA_K=2, nlevels=8, patchSize=31)
keypoints, descriptors = orb.detectAndCompute(img, None)
```

* To see what the parameters mean, check the doc: <http://docs.opencv.org/trunk/db/d95/classcv_1_1ORB.html>


1. For the template and scene images, visualize what `keypoints` and `descriptors` represent (dimensions...). 

2. To visualize the keypoints of each image, use the `cv2.drawKeypoints()` method and show the resulting image:

```python
img_with_keypoints = cv2.drawKeypoints(img, keypoints, None)
```

In [ ]:
# Open images
template = cv2.imread('beer.png',0)
scene = cv2.imread('scene.jpg',0)

# Initialize a ORB detector
orb = cv2.ORB_create(nfeatures=1500, scaleFactor=1.3, WTA_K=2, nlevels=8, patchSize=31)

# Find the keypoints and descriptors with ORB
template_keypoints, template_descriptors = orb.detectAndCompute(template, None)
scene_keypoints, scene_descriptors = orb.detectAndCompute(scene, None)

# Draw the keypoints on the images
template_with_keypoints = cv2.drawKeypoints(template, template_keypoints, None)
scene_with_keypoints = cv2.drawKeypoints(scene, scene_keypoints, None)

plt.figure()
plt.imshow(template_with_keypoints)
plt.xticks([])
plt.yticks([])
plt.title('template with keypoints')
plt.show()

plt.figure()
plt.imshow(scene_with_keypoints)
plt.xticks([])
plt.yticks([])
plt.title('scene with keypoints')
plt.show()

### **Task 3:** Feature matching

1. OpenCV provides a **brute-force matcher** based on **k-nearest neighbors** and a **FLANN** matcher based on **k-d trees**. We will use the brute force matcher here: 

```python
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = bf.knnMatch(template_descriptors, scene_descriptors, k=2)
```

* Here we get a list of the **2** nearest neighbours for each feature.

* We use the **Hamming distance**, as we have ORB feature vectors. SIFT features would only require the Euclidian distance.


2. To visualize the matches between the template and the scene, use the `cv2.drawMatchesKnn` method:

```python
image_with_matches = cv2.drawMatchesKnn(
    template, # First image
    template_keypoints, # Keypoints of the first image
    scene, # Second image
    scene_keypoints, # Keypoints of the second image
    matches, # Detected matches
    None, # Leave it
    flags=2 # What to plot
)
```

In [ ]:
# Brute Force Matcher with default params
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = bf.knnMatch(template_descriptors, scene_descriptors, k=2)

image_with_matches = cv2.drawMatchesKnn(
    template, template_keypoints,
    scene, scene_keypoints, matches, None, flags=2)

plt.figure()
plt.imshow(image_with_matches)
plt.xticks([])
plt.yticks([])
plt.show()

### **Task 4:** Keeping only "good" matches


* A match is good if and only if it is the only possible, i.e. the second nearest neighbor is far away from the first nearest.

1. Reject matches that do not fit the **Nearest neighbor distance ratio matching** criteria:

```python
good_matches = []
for first, second in matches:
    if first.distance < 0.9 * second.distance:
        good_matches.append([first])
```

2. How many good matches do you have? Print their number, and a message if you have less than 10 good matches.

3. Visualize the matches. Are the matches good enough? Are there outliers? Why?

In [ ]:
# Apply ratio test to keep only good neighbours
good_matches = []
for first, second in matches:
    if first.distance < 0.9 * second.distance:
        good_matches.append([first])
print("Found", len(good_matches), 'matches.')

# a meesage when not enough matches
if len(good_matches) < 10:
    print('Less than 10 good matches')

image_with_good_matches = cv2.drawMatchesKnn(
    template, template_keypoints, scene, scene_keypoints,
    good_matches, None, flags=2)

plt.figure()
plt.imshow(image_with_good_matches)
plt.xticks([])
plt.yticks([])
plt.show()

### **Task 5:** Feature alignment

1. The next step is to use RANSAC to estimate the projective transformation (*homography*) between the template and the scene.

In [ ]:
# Reshape the keypoints
template_points = np.float32(
    [ template_keypoints[m[0].queryIdx].pt for m in good_matches ]
                            ).reshape(-1,1,2)
scene_points = np.float32(
    [ scene_keypoints[m[0].trainIdx].pt for m in good_matches ]
                         ).reshape(-1,1,2)

# Find the homography with RANSAC
M, mask = cv2.findHomography(template_points, scene_points, cv2.RANSAC, ransacReprojThreshold=5.0)

2. Analyse the estimated projection between the two sets of points.

3. `mask` returns for all matches of `good_matches` 1 if the match is an inlier, 0 if it is an outlier. Build a new list `inliers` as a sub-list of `good_matches` where only the inliers are kept.

4. Visualize the inlier matches.

In [ ]:
print('M:\n',M)

# Find the inliers
inliers = []
for idx, match in enumerate(good_matches):
    if mask[idx, 0] == 1:
        inliers.append(match)
        
# Draw the inlier matches
image_with_inliers = cv2.drawMatchesKnn(
    template, template_keypoints, scene, scene_keypoints, inliers, None, flags=2)

plt.figure()
plt.imshow(image_with_inliers)
plt.xticks([])
plt.yticks([])
plt.show()        

### **Task 5:** Transforming borders of the template

1. Find the borders of the template:

```python
height, width = template.shape
rectangle = np.float32([ [0, 0], [0, height-1], 
                         [width-1, height-1], [width-1, 0] ]).reshape(-1,1,2)
```

2. Apply the projective transformation on it:

```python
warped = cv2.perspectiveTransform(rectangle, M)
```

3. Add the warped rectangle to a copy of the scene and visualize the result:

```python
scene2 = scene.copy()
cv2.polylines(scene2, [np.int32(warped)], True, 255, 3, cv2.LINE_AA)
```

In [ ]:
# Apply the perspective transformation on the borders of the template
height, width = template.shape
rectangle = np.float32(
    [ [0, 0], [0, height-1], [width-1, height-1], [width-1, 0] ]).reshape(-1,1,2)

warped = cv2.perspectiveTransform(rectangle, M)
scene2 = scene.copy()

cv2.polylines(scene2, [np.int32(warped)], True, 255, 3, cv2.LINE_AA)

# Draw the inlier matches
image_with_inliers = cv2.drawMatchesKnn(
    template, template_keypoints, scene2, scene_keypoints, inliers, None, flags=2)

plt.figure()
plt.imshow(image_with_inliers)
plt.xticks([])
plt.yticks([])
plt.show()     


### **Task 6 (Optional):** Try other images

1. Search for other template/scene images and search the templates in the scene. Is it a robust method?

---